In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
import tensorflow as tf

In [2]:
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

In [3]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):
    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [25]:
BATCH_SIZE=32
TARGET_SIZE=(24,24)
eyes_train_batch= generator('./dataset/eyes/train',shuffle=True, batch_size=BATCH_SIZE,target_size=TARGET_SIZE)
eyes_valid_batch= generator('dataset/eyes/valid',shuffle=True, batch_size=BATCH_SIZE,target_size=TARGET_SIZE)
yawn_train_batch= generator('./dataset/new_yawn/train',shuffle=True, batch_size=BATCH_SIZE,target_size=TARGET_SIZE)
yawn_valid_batch= generator('dataset/new_yawn/valid',shuffle=True, batch_size=BATCH_SIZE,target_size=TARGET_SIZE)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.
Found 1540 images belonging to 2 classes.
Found 511 images belonging to 2 classes.


In [18]:
eyes_SPE= len(eyes_train_batch.classes)//BATCH_SIZE
eyes_VS = len(eyes_valid_batch.classes)//BATCH_SIZE
print(eyes_SPE,eyes_VS)

yawn_SPE= len(yawn_train_batch.classes)//BATCH_SIZE
yawn_VS = len(yawn_valid_batch.classes)//BATCH_SIZE
print(yawn_SPE,yawn_VS)

38 6
48 15


In [6]:
eyes_model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])

In [7]:
eyes_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [9]:
eyes_model.fit(eyes_train_batch, validation_data=eyes_valid_batch,epochs=30,steps_per_epoch=eyes_SPE ,validation_steps=eyes_VS, callbacks=[callback])

Epoch 1/30
38/38 [==============================] - 6s 155ms/step - loss: 0.3648 - accuracy: 0.8569 - val_loss: 0.1742 - val_accuracy: 0.9427
Epoch 2/30
38/38 [==============================] - 5s 122ms/step - loss: 0.1691 - accuracy: 0.9351 - val_loss: 0.1345 - val_accuracy: 0.9479
Epoch 3/30
38/38 [==============================] - 5s 123ms/step - loss: 0.1183 - accuracy: 0.9576 - val_loss: 0.1222 - val_accuracy: 0.9635
Epoch 4/30
38/38 [==============================] - 5s 123ms/step - loss: 0.1003 - accuracy: 0.9676 - val_loss: 0.1308 - val_accuracy: 0.9479
Epoch 5/30
38/38 [==============================] - 5s 123ms/step - loss: 0.0683 - accuracy: 0.9725 - val_loss: 0.1370 - val_accuracy: 0.9531
Epoch 6/30
38/38 [==============================] - 5s 126ms/step - loss: 0.0640 - accuracy: 0.9734 - val_loss: 0.1055 - val_accuracy: 0.9740
Epoch 7/30
38/38 [==============================] - 5s 122ms/step - loss: 0.0612 - accuracy: 0.9759 - val_loss: 0.1129 - val_accuracy: 0.9635
Epoch 

In [34]:
eyes_model.save('models/cnnEyes.h5', overwrite=True)

In [12]:
img = cv2.imread('./dataset/eyes/valid/open/_359.jpg')
resized = cv2.resize(img/255, (24, 24))

In [15]:

rpred = np.argmax(eyes_model.predict(resized.reshape(-1,24, 24, 1)), axis=-1)

In [16]:
rpred

array([0, 1, 1], dtype=int64)

In [19]:
yawn_model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])

In [20]:
yawn_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
yawn_model.fit(yawn_train_batch, validation_data=yawn_valid_batch,epochs=30,steps_per_epoch=yawn_SPE ,validation_steps=yawn_VS, callbacks=[callback])

Epoch 1/30
48/48 [==============================] - 12s 242ms/step - loss: 0.3040 - accuracy: 0.8674 - val_loss: 0.0329 - val_accuracy: 0.9875
Epoch 2/30
48/48 [==============================] - 4s 74ms/step - loss: 0.0116 - accuracy: 0.9960 - val_loss: 0.0169 - val_accuracy: 0.9896
Epoch 3/30
48/48 [==============================] - 4s 73ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 0.9875
Epoch 4/30
48/48 [==============================] - 3s 71ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0155 - val_accuracy: 0.9979
Epoch 5/30
48/48 [==============================] - 3s 70ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 0.0144 - val_accuracy: 0.9917
Epoch 6/30
48/48 [==============================] - 3s 71ms/step - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0125 - val_accuracy: 0.9979
Epoch 7/30
48/48 [==============================] - 3s 70ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0187 - val_accuracy: 0.9875


In [27]:
yawn_model.save('models/cnnYawn.h5', overwrite=True)

In [10]:
import cv2

In [24]:
loaded_model = load_model('models/cnnYawn.h5')

In [26]:
yawn_model.fit(yawn_train_batch, validation_data=yawn_valid_batch,epochs=30,steps_per_epoch=yawn_SPE ,validation_steps=yawn_VS, callbacks=[callback])

Epoch 1/30
48/48 [==============================] - 4s 76ms/step - loss: 0.0050 - accuracy: 0.9980 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 2/30
48/48 [==============================] - 4s 74ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0166 - val_accuracy: 0.9979
Epoch 3/30
48/48 [==============================] - 4s 75ms/step - loss: 9.6904e-04 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9979
Epoch 4/30
48/48 [==============================] - 4s 76ms/step - loss: 5.0592e-04 - accuracy: 1.0000 - val_loss: 0.0218 - val_accuracy: 0.9979
Epoch 5/30
48/48 [==============================] - 4s 73ms/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.0278 - val_accuracy: 0.9917
Epoch 6/30
48/48 [==============================] - 4s 73ms/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.0046 - val_accuracy: 0.9958
Epoch 7/30
48/48 [==============================] - 4s 75ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 0.9979


In [29]:
img = cv2.imread('img.png')
resized = cv2.resize(img/255, (24, 24))
loaded_model.predict(resized.reshape(-1,24, 24, 1))
prediction = np.argmax(loaded_model.predict(resized.reshape(-1,24, 24, 1)))

array([[4.1551814e-12, 1.0000000e+00],
       [1.1281472e-02, 9.8871851e-01],
       [1.9583416e-05, 9.9998045e-01]], dtype=float32)